In [1]:
%%capture
%pip install -U datapackage jupysql duckdb-engine --quiet

In [2]:
import duckdb
import datapackage

In [3]:
con = duckdb.connect(":memory:")

con.sql("""
    install httpfs;
    load httpfs;
"""
)

In [4]:
def datapackage_to_sql(datapackage_url: str, con: duckdb.DuckDBPyConnection):
    package = datapackage.Package(datapackage_url)
    for r in package.resources:
        if r.tabular:
            con.sql(
                f"""
            create or replace table {r.name.replace('-','_')} as select * from read_csv_auto('{r.source}', HEADER=TRUE);
            """
            )

In [6]:
datapackage_to_sql("https://raw.githubusercontent.com/datasets/un-locode/master/datapackage.json", con)
datapackage_to_sql('https://datahub.io/core/population/datapackage.json', con)

In [10]:
con.sql("show tables")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ code_list            │
│ country_codes        │
│ function_classifiers │
│ population           │
│ population_csv       │
│ status_indicators    │
│ subdivision_codes    │
└──────────────────────┘

In [24]:
con.sql("""
select 
    "Country Code",
    "Name",
    "Value" as Population,
from code_list left join population on code_list.Location == population."Country Code" limit 10
""")

┌──────────────┬───────────────────────────┬────────────┐
│ Country Code │           Name            │ Population │
│   varchar    │          varchar          │   int64    │
├──────────────┼───────────────────────────┼────────────┤
│ CAN          │ Canillo                   │   37057765 │
│ ALB          │ Al Barsha                 │    2866376 │
│ IND          │ Indooroodilly             │ 1352617328 │
│ IDA          │ Jebel Ali Industrial Area │ 1640238121 │
│ SWE          │ Swaihan                   │   10175214 │
│ FRO          │ FOB Frontenac             │      48497 │
│ KWT          │ Khowst                    │    4137309 │
│ SOM          │ Sombrero                  │   15008154 │
│ ARG          │ Argyrokastro              │   44494502 │
│ ARG          │ Aragatsotn                │   44494502 │
├──────────────┴───────────────────────────┴────────────┤
│ 10 rows                                     3 columns │
└───────────────────────────────────────────────────────┘